<a href="https://colab.research.google.com/github/gpasxos/large-scale-optimization/blob/main/ch06_newton_method.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Newton's Method with Backtracking Line Search

Newton's method updates the iterate according to:
$$x^{(k+1)} = x^{(k)} + \eta^{(k)} \Delta x_{\text{nt}}$$

where the **Newton step** is:
$$\Delta x_{\text{nt}} = -[\nabla^2 f(x^{(k)})]^{-1} \nabla f(x^{(k)})$$

### Newton Decrement

The **Newton decrement** provides a stopping criterion:
$$\lambda(x) = \sqrt{\nabla f(x)^T [\nabla^2 f(x)]^{-1} \nabla f(x)}$$

We stop when $\lambda^2/2 \leq \epsilon$, since this approximates the function decrease from one Newton step.

### Backtracking Line Search

We use **Armijo backtracking**: start with $\eta = 1$ and shrink by factor $\beta$ until:
$$f(x + \eta \Delta x_{\text{nt}}) \leq f(x) - \alpha \eta \lambda^2$$

## Demonstration: Ill-Conditioned Quadratic

We test on a quadratic $f(x) = \frac{1}{2} x^T Q x + b^T x$ where $Q$ has condition number $\kappa = 1000$.

**Why this is a challenging test case:**
- High condition number severely slows gradient descent (rate $\approx 1 - 1/\kappa$)
- Newton's affine invariance means $\kappa$ doesn't affect its convergence

## Key Takeaways

| Method | Iterations | Per-iteration cost | Total cost |
|--------|------------|-------------------|------------|
| Newton | $O(\log\log(1/\epsilon))$ | $O(n^3)$ | Low |
| Gradient Descent | $O(\kappa \log(1/\epsilon))$ | $O(n^2)$ | High when $\kappa$ large |

1. **Quadratic convergence**: Once near the optimum, Newton doubles correct digits each iteration
2. **Affine invariance**: Newton's performance is independent of condition number $\kappa$
3. **Practical regime**: Prefer Newton when $n \lesssim 10^4$ and high accuracy is needed

In [1]:
import numpy as np
from scipy.linalg import cho_factor, cho_solve
def newton_method(f, grad_f, hess_f, x0, tol=1e-10, max_iter=100, alpha=0.25, beta=0.5, verbose=True):
    """
    Newton's method with backtracking line search.
    Parameters:
        f: objective function
        grad_f: gradient function
        hess_f: Hessian function (returns n x n matrix)
        x0: initial point
        tol: tolerance for Newton decrement squared / 2
        alpha: Armijo parameter (0 < alpha < 0.5)
        beta: backtracking shrinkage factor (0 < beta < 1)
        verbose: print progress

    Returns:
        x: optimal point
        history: dict with convergence information
    """
    x = np.array(x0, dtype=float)
    n = len(x)

    history = {
        'f': [f(x)],
        'newton_decrement': [],
        'step_size': [],
        'x': [x.copy()]
    }

    for k in range(max_iter):
        # Compute gradient and Hessian
        g = grad_f(x)
        H = hess_f(x)

        # Solve H @ dx = -g for Newton step using Cholesky factorization
        try:
            c, lower = cho_factor(H)
            dx = cho_solve((c, lower), -g)
        except np.linalg.LinAlgError:
            print("Hessian not positive definite, adding regularization")
            H_reg = H + 1e-6 * np.eye(n)
            c, lower = cho_factor(H_reg)
            dx = cho_solve((c, lower), -g)

        # Compute Newton decrement
        lambda_sq = -g @ dx # = g^T H^{-1} g
        newton_decrement = np.sqrt(lambda_sq)
        history['newton_decrement'].append(newton_decrement)

        # Check stopping criterion
        if lambda_sq / 2 <= tol:
            if verbose:
                print(f"Converged at iteration {k}: lambda^2/2 = {lambda_sq/2:.2e}")
            break

        # Backtracking line search
        eta = 1.0
        f_x = f(x)
        while f(x + eta * dx) > f_x - alpha * eta * lambda_sq:
            eta *= beta
            if eta < 1e-16:
                print("Warning: step size too small")
                break

        history['step_size'].append(eta)

        # Update
        x = x + eta * dx
        history['f'].append(f(x))
        history['x'].append(x.copy())

        if verbose:
            print(f"Iter {k:3d}: f = {f(x):.6e}, lambda = {newton_decrement:.2e}, eta = {eta:.4f}")

    return x, history

def demo_newton():
    """Demonstrate Newton's method on an ill-conditioned quadratic."""
    np.random.seed(42)
    n = 20

    # Create ill-conditioned quadratic: f(x) = 0.5 * x^T Q x + b^T x
    kappa = 1000 # Condition number
    eigenvalues = np.linspace(1, kappa, n)
    Q_diag = np.diag(eigenvalues)

    # Random rotation
    U, _ = np.linalg.qr(np.random.randn(n, n))
    Q = U @ Q_diag @ U.T
    b = np.random.randn(n)

    # Optimal solution
    x_star = np.linalg.solve(Q, -b)
    f_star = 0.5 * x_star @ Q @ x_star + b @ x_star

    f = lambda x: 0.5 * x @ Q @ x + b @ x
    grad_f = lambda x: Q @ x + b
    hess_f = lambda x: Q

    # Initial point
    x0 = np.random.randn(n) * 10

    print("="*60)
    print("Newton's Method on Ill-Conditioned Quadratic")
    print(f"Dimension: {n}, Condition number: {kappa}")
    print("="*60)

    x_newton, history = newton_method(f, grad_f, hess_f, x0, verbose=True)

    print(f"\nFinal error: ||x - x*|| = {np.linalg.norm(x_newton - x_star):.2e}")
    print(f"Final objective gap: f(x) - f* = {f(x_newton) - f_star:.2e}")

    # Compare with gradient descent
    print("\n" + "="*60)
    print("Gradient Descent for comparison")
    print("="*60)

    x_gd = x0.copy()
    eta_gd = 1.0 / kappa # Step size = 1/L
    gd_errors = [np.linalg.norm(x_gd - x_star)]

    for k in range(1000):
        x_gd = x_gd - eta_gd * grad_f(x_gd)
        gd_errors.append(np.linalg.norm(x_gd - x_star))
        if gd_errors[-1] < 1e-10:
            break

    print(f"GD converged in {len(gd_errors)-1} iterations")
    print(f"Newton converged in {len(history['f'])-1} iterations")

    return history, gd_errors

# Run demonstration
history, gd_errors = demo_newton()

Newton's Method on Ill-Conditioned Quadratic
Dimension: 20, Condition number: 1000
Iter   0: f = -5.756522e-01, lambda = 1.36e+03, eta = 1.0000
Converged at iteration 1: lambda^2/2 = 4.24e-25

Final error: ||x - x*|| = 9.00e-13
Final objective gap: f(x) - f* = -1.71e-14

Gradient Descent for comparison
GD converged in 1000 iterations
Newton converged in 1 iterations
